# Conver Vector to Raster with reference coordinate system from raster tiff

all these tools made for prepare data for paddlepaddlea.

In [ ]:
from osgeo import gdal, gdal_array, ogr
import mygrid as cropfarm

- fn_ras = Input raster data (GTiff)
- fn_vec = input vector data (Shapefile)

In [ ]:
fn_ras = 'DataSet/raster/2019_9_4_res.tif'
fn_vec = 'DataSet/vector/esribbb.shp'
output = 'DataSet/raster/lab_all_values.tif'
att_field = 'fid' # attribute field in vector table like building id

In [ ]:
ras_ds = gdal.Open(fn_ras)
vec_ds = ogr.Open(fn_vec)

In [ ]:
lyr = vec_ds.GetLayer()
geot = ras_ds.GetGeoTransform()
proj = ras_ds.GetProjection() # Get the projection from original tiff (fn_ras)

In [ ]:
drv_tiff = gdal.GetDriverByName("GTiff")
chn_ras_ds = drv_tiff.Create(output, ras_ds.RasterXSize, ras_ds.RasterYSize, 1, gdal.GDT_Byte)
chn_ras_ds.SetGeoTransform(geot)

In [ ]:
gdal.RasterizeLayer(chn_ras_ds, [1], lyr, options=['ATTRIBUTE=fid'])
chn_ras_ds.GetRasterBand(1).SetNoDataValue(0.0) # Change No Data Value to 0
chn_ras_ds.SetProjection (proj) # Set the projection from original tiff (fn_ras) to the rasterized tiff
chn_ras_ds = None

Change all Values >= 1 to 1

In [ ]:
final_output = "DataSet/raster/final_lab.tif"

In [ ]:
ds = gdal.Open(output)
ds.GetRasterBand(1).SetNoDataValue(-9999)
ds.FlushCache()

In [ ]:
# create color table
# colors = gdal.ColorTable()

# set color for the only value
# colors.SetColorEntry(1, (225, 89, 137))

# set color table and color interpretation
# ds.SetRasterColorTable(colors)
# ds.SetRasterColorInterpretation(gdal.GCI_PaletteIndex)

In [ ]:
arr = ds.ReadAsArray()
data = (arr >= 1)
gdal_array.SaveArray(data.astype("byte"), final_output, "GTIFF", ds)
data = None

In [ ]:
cropfarm.mygridfun(fn_ras, "jpg","DataSet/image/image", "JPEG", "" ) #Sat_Raster
cropfarm.mygridfun(final_output, "png", "DataSet/label/label", "PNG", "" ) #label_raster